In [1]:
import plotly.express as px
import lostructpy.lostruct as ls
import numpy as np
from skbio.stats.ordination import pcoa
from sklearn.manifold import MDS
import pandas as pd
import umap
import hdbscan
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
windows, positions = ls.parse_vcf("chr1-filtered.vcf.gz", "chr1", 95)

In [3]:
samples = ls.get_samples("chr1-filtered.vcf.gz")

In [4]:
weights = np.random.random_sample(len(samples))
weights

array([0.97687061, 0.51224707, 0.86850825, 0.33884153, 0.9928695 ,
       0.38564121, 0.14129846, 0.1772007 , 0.68423994, 0.30382249,
       0.04223043, 0.14132182, 0.18547745, 0.86437096, 0.88066354,
       0.54097727, 0.41345157, 0.81748838, 0.98462578, 0.4862412 ,
       0.07502484, 0.04134619, 0.94547716, 0.12336358, 0.97864831,
       0.36693943, 0.67314675, 0.52501179, 0.11622793, 0.48505998,
       0.18669364, 0.49367241, 0.26351858, 0.1615667 , 0.56883563,
       0.45243737, 0.29583367, 0.90843192, 0.72261889, 0.02374371,
       0.20181461, 0.34121835, 0.9716366 , 0.08361673, 0.6018823 ,
       0.57970354, 0.40444953, 0.53154295, 0.5848248 , 0.91876757])

In [5]:
result = list()
for x in windows:
    result.append(ls.eigen_windows(x, 10, weights))
result = np.vstack(result)
pc_dists = ls.get_pc_dists(result)
mds = pcoa(pc_dists)
mds_likelostructr = mds


ValueError: operands could not be broadcast together with shapes (50,50) (2500,) 

In [ ]:
mds_coords = pd.read_csv("lostruct-results/mds_coords.csv")
np.corrcoef(mds.samples['PC1'], mds_coords['MDS1'].to_numpy())

In [ ]:
px.scatter(x=mds.samples["PC1"], y=mds_coords['MDS1'])

In [ ]:
px.scatter(y=mds.samples["PC1"])

In [ ]:
px.scatter(y=mds.samples["PC2"])

# Some looks at other methods of clustering / comparing

In [ ]:
embedding = MDS(n_components=10, dissimilarity="precomputed", n_jobs=-1, n_init=32)
mds = embedding.fit_transform(pc_dists)
px.scatter(y=[mds[:,0], mds_coords['MDS1']])

In [ ]:
import phate
phater = phate.PHATE(n_components=10, knn_dist='precomputed', mds_solver='smacof', mds='metric')
comparison_phate = phater.fit_transform(pc_dists)

In [ ]:
px.scatter(y=[mds_coords['MDS1'], mds_likelostructr.samples["PC1"], comparison_phate[:,0]])

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(pc_dists)
px.scatter(x=embedding[:, 0], y=embedding[:, 1])

In [ ]:
hdbscan_labels = hdbscan.HDBSCAN().fit_predict(embedding)
px.scatter(x=embedding[:, 0], y=embedding[:, 1], color=hdbscan_labels)

In [ ]:
reducer = umap.UMAP(n_components=3)
embedding = reducer.fit_transform(pc_dists)
hdbscan_labels = hdbscan.HDBSCAN().fit_predict(embedding)
fig = px.scatter_3d(x=embedding[:, 0], y=embedding[:, 1], z=embedding[:, 2], color=hdbscan_labels, width=800, height=600)
fig.show()

In [ ]:
result[0][0]

In [ ]:
testmat = np.matrix([[1, 4, 7], [2,5,8], [3,6,9]])
testmat

In [ ]:
sqrt_w = np.sqrt(np.array([1,2,4]))[np.newaxis]
sqrt_w

In [ ]:
sqrt_w * testmat

In [ ]:
sqrt_w[np.newaxis].T

In [ ]:
testmat * np.tile(sqrt_w, (3, 1))

In [ ]:
x = np.multiply(testmat, sqrt_w.T)
x

In [ ]:
np.multiply(x, sqrt_w)